In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import random
import math

import matplotlib.pyplot as plt
from shapely import Point

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import warnings

warnings.filterwarnings('ignore')

**Data**: https://www.kaggle.com/datasets/sumaiaparveenshupti/los-angeles-crime-data-20102020

In [ ]:
recent = pd.read_csv(r"C:\Users\Harsh Shah\Desktop\Crime_Data_from_2010_to_2019.csv")[['LAT', 'LON']]
old = pd.read_csv(r"C:\Users\Harsh Shah\Desktop\Crime_Data_from_2020_to_Present.csv")[['LAT', 'LON']]
df = pd.concat([old, recent])
df = df[(df.LON!=0) & (df.LAT!=0)]
df

In [ ]:
geometry = []
for lat, lon in zip(df.LAT, df.LON):
  geometry.append(Point(lon, lat))

gdf = gpd.GeoDataFrame(df, geometry = geometry)
gdf.plot()

In [ ]:
def coords(x,y, base=0.01):
	x, y = round(base * math.ceil(abs(x)/base),2), round(base * math.ceil(y/base),2)
	return (y,x)

def NN(data, LAT, LON):
  array = np.zeros((LAT.shape[0], LON.shape[0]),dtype=int)
  onGrid = data.apply(lambda row: coords(row.LAT, row.LON, 0.01), axis = 1).value_counts()
  for coor in onGrid.index:
    lon_idx, lat_idx = np.where(LON==coor[0]), np.where(LAT==coor[1])
    array[lat_idx,lon_idx] = int(onGrid[coor])
  return array

In [ ]:
LAT, LON = np.arange(round(df.LAT.min()), round(df.LAT.max()), 0.01).astype(np.float32), np.arange(round(df.LON.min()), round(df.LON.max()), 0.01).astype(np.float32)
crimes = NN(df, LAT, LON)
ds = xr.Dataset(
    {'Crimes': (['lat', 'lon'], crimes)},
    coords={'lat': LAT, 'lon': LON})
ds

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()), figsize=(16, 9))

ds.Crimes.plot(ax=ax, cmap='Reds')
ax.set_extent([-118.9, -118.1, 33.6, 34.5 ], crs=ccrs.PlateCarree())
ax.gridlines(draw_labels=True,linewidth=2, color='black', alpha=0.5, linestyle='--')
ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=1)

ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=1)
ax.add_feature(cartopy.feature.RIVERS, edgecolor='blue', linewidth=0.5)
states_provinces = cfeature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces',
            scale='10m', facecolor='none')

plt.show()

# PyDeck

In [ ]:
!pip install pydeck

In [ ]:
import pydeck as pdk

https://deck.gl/docs/api-reference/layers

# 1. Hexagons

In [ ]:
layer = pdk.Layer(
    'HexagonLayer',
    df,
    get_position=['LON', 'LAT'],
    radius=500, #bin radius
    auto_highlight=True,
    elevation_scale=50, #scale factor for bins (the greater - the higher)
    elevation_range=[0, 3000],
    pickable=True,
    extruded=True,#cell elevation
    )

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('hex.html')

# 2. Columns

In [ ]:
layer = pdk.Layer(
    'ColumnLayer',
    ds.to_dataframe().reset_index(),
    get_position=['lon', 'lat'],
    get_elevation='Crimes',
    elevation_scale=10,
    radius=200,
    get_fill_color=['Crimes', 220],
    pickable=True,
    extruded=True,
    )

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html('column.html')

# 3. Scatter plots

In [ ]:
layer = pdk.Layer(
    'ColumnLayer',
    df[:15000],
    get_position=['LON', 'LAT'],
    auto_highlight=True,
    get_radius=200,          # Radius is given in meters
    get_fill_color=[180, 0, 200, 140],  # Set an RGBA value for fill
    pickable=True)

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r#.to_html('scatter.html')

# 4. Map style

In [ ]:
 layer = pdk.Layer(
    'HexagonLayer',
    df[:1500],
    get_position=['LON', 'LAT'],
    radius=500, #bin radius
    auto_highlight=True,
    elevation_scale=50, #scale factor for bins (the greater - the higher)
    elevation_range=[0, 3000],
    pickable=True,
    extruded=True,#cell elevation
    )

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

r = pdk.Deck(layers=[layer],
             initial_view_state=view_state,
             map_style=pdk.map_styles.LIGHT, # ‘light’, ‘dark’, ‘road’, ‘satellite’, ‘dark_no_labels’, and ‘light_no_labels
             )
r#.to_html('hex.html')

# Custom titles
https://deckgl.readthedocs.io/en/latest/tooltip.html

In [ ]:

 layer = pdk.Layer(
    'HexagonLayer',
    df[:1500],
    get_position=['LON', 'LAT'],
    radius=500, #bin radius
    auto_highlight=True,
    elevation_scale=50, #scale factor for bins (the greater - the higher)
    elevation_range=[0, 3000],
    pickable=True,
    extruded=True,#cell elevation
    )

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

r = pdk.Deck(layers=[layer],
             initial_view_state=view_state,
             map_style=pdk.map_styles.DARK, # ‘light’, ‘dark’, ‘road’, ‘satellite’, ‘dark_no_labels’, and ‘light_no_labels
             tooltip = {
                  "html": "<b>Number of crimes:</b> {elevationValue}",
                  "style": {
                        "backgroundColor": "yellow",
                        "color": "black"
                  }
                },
             )
r#.to_html('hex.html')

# Interactive angle
https://deckgl.readthedocs.io/en/latest/event_handling.html

In [ ]:
from ipywidgets import HTML

text = HTML(value='Move the viewpoint')

layer = pdk.Layer(
    'HexagonLayer',
    df,
    get_position=['LON', 'LAT'],
    radius=500, #bin radius
    auto_highlight=True,
    elevation_scale=50, #scale factor for bins (the greater - the higher)
    elevation_range=[0, 3000],
    pickable=True,
    extruded=True,#cell elevation
    )

view_state = pdk.ViewState(
    longitude=-118.3,
    latitude=34.4,
    zoom=8,
    min_zoom=6,
    max_zoom=15,
    bearing=-20,#left/right angle
    pitch=20, #up/down angle
    )

r = pdk.Deck(layers=[layer],
             initial_view_state=view_state,
             map_style=pdk.map_styles.DARK)

def filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat):
    return west_lng < row['lng'] < east_lng and south_lat < row['lat'] < north_lat

def filter_by_viewport(widget_instance, payload):
    west_lng, north_lat = payload['data']['nw']
    east_lng, south_lat = payload['data']['se']
    filtered_df = df[df.apply(lambda row: filter_by_bbox(row, west_lng, east_lng, north_lat, south_lat), axis=1)]


r.deck_widget.on_click(filter_by_viewport)
r.to_html('angle.html')